In [ ]:
!pip install transformers # transformers 설치
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv',delimiter='\t', header=None) # 데이터 불러오기

batch_1 = df[:2000] # 배치
print(batch_1.head(3))

print(batch_1[1].value_counts( ))

     |████████████████████████████████| 2.3MB 25.3MB/s 
     |████████████████████████████████| 901kB 38.7MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 
                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
1    1041
0     959
Name: 1, dtype: int64


In [ ]:
model_class = ppb.BertModel # 버트 모델 객체 생성
tokenizer_class = ppb.BertTokenizer # 워드 피스  Byte-pair encoding 방식으로 주어진 데이터에서 사전을 학습
pretrained_weights = 'bert-base-uncased'

# 사전학습된 model/tokenizer 적재
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(tokenized)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) # 패딩
print(padded)

attention_mask = np.where(padded != 0, 1, 0) # 패딩 마스크
print(attention_mask.shape)

input_ids = torch.tensor(padded) # 텐서
attention_mask = torch.tensor(attention_mask) # 텐서

with torch.no_grad( ):
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

[[  101  1037 18385 ...     0     0     0]
 [  101  4593  2128 ...     0     0     0]
 [  101  2027  3653 ...     0     0     0]
 ...
 [  101  2023  2028 ...     0     0     0]
 [  101  1999  1996 ...     0     0     0]
 [  101  1996  3185 ...     0     0     0]]
(2000, 59)


In [ ]:
features = last_hidden_states[0][:,0,:].numpy( ) # 문장에 대한 특징벡터
labels = batch_1[1]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels) # 데이터 분리

lr_clf = LogisticRegression( ) # 로지스트 회귀
lr_clf.fit(train_features, train_labels) # 학습

print('accuracy : ', lr_clf.score(test_features, test_labels)) # 정확도

accuracy :  0.826
